<a href="https://colab.research.google.com/github/Entropicsky/Auto-GPT/blob/master/Multi_Question_Free_Form_Text_Survey_Answer_Analysis_For_Distro_BYO_API_Key.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOTE: YOU NEED TO PUT IN YOUR OPENAI API KEY!!!!!!!**

This notebook imports an Excel file that should contain "N" columns of free-form text survey inputs:

Column 1: Respondent Id

Remaining Columns: The column header should be the question that was asked, and then each row should be the free-form text survey responses.

The HEADER on Column 2 to N should be the actual question(s) that were asked. For example: What are things you know now that you wish you knew when you started playing?

The notebook will then rip through that file, evaluate each qeustion one-by-one, chunking it, summarizing it and ultimately creating and downloading a word document that summarizes the answers from that question for you (the name of that document will be the question name, scrubbed of weird characters).

Finally, the program will create one unique word document that summarizes all of the questions in the document (but without the same level of detail as the question-specific documents).


STEP 1: Install the necessary Python Libraries

In [ ]:
!pip install openai pandas nltk
!pip install python-docx  # install the python-docx library


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


STEP 2: Prompt the user to upload the Excel file. **NOTE: YOU NEED TO PUT IN YOUR OPENAI API KEY!!!!!!!**

In [ ]:
import openai
import pandas as pd
import nltk
import time
from google.colab import files
from openai.error import OpenAIError, RateLimitError
from docx import Document

# Initialize OpenAI API
openai.api_key = '[YOUR KEY]'

# Prompt the user to upload a file
uploaded = files.upload()

Saving SMITE UXR Survey April 2023 Olympians Pro Players  Experienced MOBA players.xls to SMITE UXR Survey April 2023 Olympians Pro Players  Experienced MOBA players.xls


STEP 3: The load the Excel file into a dataframe, chunk it, and use the ChatGPT model to summarize each chunk, then summarize the summaries. Create word docs from this.

In [ ]:

# Assuming 'uploaded' is a dictionary of uploaded files
for fn in uploaded.keys():
    data = pd.read_excel(fn)

all_summaries_dict = {}

# Iterate over each question column (skipping the Respondent Id column)
for question in data.columns[1:]:
    # Concatenate all feedbacks into a single string for the current question
    all_feedbacks = ' '.join(str(x) for x in data[question].dropna())

    # Split the string into ~5000 character chunks
    chunks = [all_feedbacks[i:i+5000] for i in range(0, len(all_feedbacks), 5000)]

    # List to store summaries of each chunk
    chunk_summaries = []

    for chunk in chunks:
        try:
            # Call the OpenAI API to summarize the chunk
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",
                messages=[
                    {"role": "system", "content": "You are a marketing analyst for a video game called SMITE. Your job is to read survey data and prepare well formatted, thorough summaries in bulleted form, so that the designers of the game can improve the game."},
                    {"role": "user", "content": f"Please summarize the following player responses to the question: '{question}'. Break the responses into 5 different categories and then provide bulleted summaries of feedback for each category. Try to do around 5 bullets per category. Here are the responses: {chunk}"}
                ]
            )
            # Store the summary
            chunk_summaries.append(response['choices'][0]['message']['content'])
        except RateLimitError:
            print("Rate limit exceeded. Sleeping for 60 seconds.")
            time.sleep(60)  # Wait for 60 seconds before retrying

    # Now summarize the summaries
    all_summaries = ' '.join(chunk_summaries)

    try:
        # Call the OpenAI API to summarize all summaries
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[
                {"role": "system", "content": "You are a marketing analyst for a video game called SMITE. Your job is to read survey data and prepare well formatted, thorough summaries in bulleted form, so that the designers of the game can improve the game."},
                {"role": "user", "content": f"Please summarize the following summaries of player responses to the question: '{question}'. Break the responses into 5 different categories and then provide bulleted summaries of feedback for each category. Try to do around 5 bullets per category. Here are the summaries: {all_summaries}"}
            ]
        )
        all_summaries = response['choices'][0]['message']['content']
        all_summaries_dict[question] = all_summaries  # Save summary for all summaries doc
    except RateLimitError:
        print("Rate limit exceeded. Sleeping for 60 seconds.")
        time.sleep(60)  # Wait for 60 seconds before retrying




    # Create a Word document
    filename = ''.join(e for e in question if e.isalnum())  # Scrub weird characters
    doc = Document()
    doc.add_heading('Question', 0)
    doc.add_paragraph(question)
    doc.add_heading('Overall Summary', 0)
    doc.add_paragraph(all_summaries)
    doc.add_heading('Chunk Summaries', 0)
    for i, summary in enumerate(chunk_summaries, start=1):
        doc.add_heading(f'Chunk {i}', level=1)
        doc.add_paragraph(summary)
    doc.add_heading('Detail', 0)
    for feedback in data[question].dropna():
        doc.add_paragraph("* " + str(feedback))
    doc.save(f'{filename}.docx')

    files.download(filename + '.docx')

# Create a single Word document with overall summaries for all questions
doc = Document()
for question, summary in all_summaries_dict.items():
    doc.add_heading(question, 0)
    doc.add_paragraph(summary)
doc.save('all_summaries.docx')
files.download('all_summaries.docx')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>